In [144]:
# https://plot.ly/python/subplots/
import plotly as py
import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs

# Когда выгоднее использовать быструю, а когда  - медленную в зависимости от длины входных данных
# Корелляция - все то же самое

py.offline.init_notebook_mode(connected=True)

# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
from sklearn.metrics import mean_squared_error

def print_mse(got, predicted, prefix=''):
    mse = mean_squared_error(got, predicted)
    if isinstance(mse, np.complex):
        print('%s Mean Squared Error: (%.2g, %.2g)' % (prefix, mse.real, mse.imag))
    else:
        print('%s Mean Squared Error: %.2g' % (prefix, mse))

def correlate(s1, s2):
    nOut = len(s1) + len(s2) - 1

    s1_padded = np.pad(s1, (0, nOut - len(s1)), 'constant', constant_values=(0,0))
    s2_padded = np.pad(s2, (nOut - len(s2), 0), 'constant', constant_values=(0,0))
    return [sum(s1_padded[m] * s2_padded[n + m] for m in range(nOut - n)) for n in range(nOut)]

def fcorrelate(s1, s2):
    nOut = len(s1) + len(s2) - 1

    s1_padded = np.pad(s1, (0, nOut - len(s1)), 'constant', constant_values=(0,0))
    s2_padded = np.pad(s2, (0, nOut - len(s2)), 'constant', constant_values=(0,0))
    
    fft1 = np.fft.fft(s1_padded)
    fft2 = np.fft.fft(s2_padded)
    return np.fft.fftshift((np.fft.ifft(np.conj(fft1) * fft2)))

In [148]:
N = 500
T = 1.0 # s
f1 = 5 # Hz
f2 = 5 # Hz
w1 = 2 * np.pi * f1
w2 = 2 * np.pi * f2

t = np.linspace(0, T, N)
t_ext = np.linspace(-T, T, 2 * N - 1)
s1 = np.cos(w1 * t / T)
s2 = np.cos(w2 * t / T)

cross = correlate(s1, s2)
fcross = fcorrelate(s1, s2)
npcross = np.correlate(s1, s2, mode='full')[::-1]

print_mse(cross, fcross, 'cross/fcross')
print_mse(cross, npcross, 'cross/npcross')
print_mse(fcross, npcross, 'fcross/npcross')

tr_cross = go.Scatter(
    x = t_ext,
    y = cross,
    name='cross'
)

tr_fcross = go.Scatter(
    x = t_ext,
    y = fcross.real,
    name='fcross'
)

tr_npcross = go.Scatter(
    x = t_ext,
    y = npcross.real,
    name='numpy'
)

fig = py.tools.make_subplots(rows = 1, cols = 1)

fig['layout'].update(height=450, width=900)

fig.append_trace(tr_cross, 1, 1)
fig.append_trace(tr_fcross, 1, 1)
fig.append_trace(tr_npcross, 1, 1)

py.offline.iplot(fig)

cross/fcross Mean Squared Error: (5.6e-26, 2.8e-26)
cross/npcross Mean Squared Error: 1.9e-27
fcross/npcross Mean Squared Error: (5.4e-26, 2.9e-26)
This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [146]:
N = 500
T = 1.0 # s

t = np.linspace(0, T, N)
t_ext = np.linspace(-T, T, 2 * N - 1)
s1 = [0 if i < 50 or i > 150 else 1 for i in range(N)]
s2 = [0 if i < 200 or i > 300 else -1 for i in range(N)]

cross = correlate(s1, s2)
fcross = fcorrelate(s1, s2)
npcross = np.correlate(s1, s2, mode='full')[::-1]

print_mse(cross, fcross, 'cross/fcross')
print_mse(cross, npcross, 'cross/npcross')
print_mse(fcross, npcross, 'fcross/npcross')

tr_signal1 = go.Scatter(
    x = t,
    y = s1,
    name='s1'
)

tr_signal2 = go.Scatter(
    x = t,
    y = s2,
    name='s2'
)

tr_conv = go.Scatter(
    x = t_ext,
    y = cross,
    name='cross'
)

tr_fconv = go.Scatter(
    x = t_ext,
    y = fcross.real,
    name='fcross'
)

tr_npconv = go.Scatter(
    x = t_ext,
    y = npcross.real,
    name='numpy'
)

fig = py.tools.make_subplots(rows = 2, cols = 1)

fig['layout'].update(height=900, width=900)

fig.append_trace(tr_signal1, 1, 1)
fig.append_trace(tr_signal2, 1, 1)
fig.append_trace(tr_conv, 2, 1)
fig.append_trace(tr_fconv, 2, 1)
fig.append_trace(tr_npconv, 2, 1)

py.offline.iplot(fig)

cross/fcross Mean Squared Error: (-1.9e-28, 1e-27)
cross/npcross Mean Squared Error: 0
fcross/npcross Mean Squared Error: (-1.9e-28, 1e-27)
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [ ]:
N = 500
T = 1.0 # s

t = np.linspace(0, T, N)
t_ext = np.linspace(-T, T, 2 * N - 1)
s1 = [0 if i < 50 or i > 150 else 1 for i in range(N)]
s2 = [0 if i < 200 or i > 300 else -1 for i in range(N)]

cross = correlate(s1, s2)
fcross = fcorrelate(s1, s2)
npcross = np.correlate(s1, s2, mode='full')[::-1]

print_mse(cross, fcross, 'cross/fcross')
print_mse(cross, npcross, 'cross/npcross')
print_mse(fcross, npcross, 'fcross/npcross')

tr_signal1 = go.Scatter(
    x = t,
    y = s1,
    name='s1'
)

tr_signal2 = go.Scatter(
    x = t,
    y = s2,
    name='s2'
)

tr_conv = go.Scatter(
    x = t_ext,
    y = cross,
    name='cross'
)

tr_fconv = go.Scatter(
    x = t_ext,
    y = fcross.real,
    name='fcross'
)

tr_npconv = go.Scatter(
    x = t_ext,
    y = npcross.real,
    name='numpy'
)

fig = py.tools.make_subplots(rows = 2, cols = 1)

fig['layout'].update(height=900, width=900)

fig.append_trace(tr_signal1, 1, 1)
fig.append_trace(tr_signal2, 1, 1)
fig.append_trace(tr_conv, 2, 1)
fig.append_trace(tr_fconv, 2, 1)
fig.append_trace(tr_npconv, 2, 1)

py.offline.iplot(fig)